In [3]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install -qq langdetect
from langdetect import detect
import string
!pip install -qq contractions
!pip install -qq demoji
import demoji
demoji.download_codes()
!pip install -qq datasets

<ipython-input-1-28a5514cb77a>:7: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [2]:
from datasets import load_dataset

dataset = load_dataset("offenseval_dravidian", "malayalam")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/16010 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1999 [00:00<?, ? examples/s]

In [4]:
df_train = pd.DataFrame(dataset["train"])
label_mapping = {
    0: 'Not_offensive',
    1: 'Offensive',
    2: 'Offensive',
    3: 'Offensive',
    4: 'Offensive',
    5: 'not-Malayalam',
}
df_train['label'].replace(label_mapping, inplace=True)

label_mapping1 = {
    'Not_offensive':0,
    'Offensive':1,
    'not-Malayalam':2,
}
df_train['label'].replace(label_mapping1, inplace=True)

print(df_train.info())
label_counts = df_train['label'].value_counts()
print(label_counts)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16010 entries, 0 to 16009
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    16010 non-null  object
 1   label   16010 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 250.3+ KB
None
0    14153
2     1287
1      570
Name: label, dtype: int64


In [5]:
df_train.head(20)

,text,label
0,പലദേശം. പല ഭാഷ ഒരേ ഒരു രാജാവ് അല്ലാതെ സ്വന്ത...,0
1,ഈ ഓണം ഏട്ടനും പിള്ളേർക്ക് ഉള്ളതാണ് എന്ന് ഉള്ളവ...,0
2,ആരണ്ട ആരണ്ട തലുണ്ടാകാണാ ആരണ്ട ഞാൻ ആണ്ട ഞാൻ ആണ്...,0
3,Sushin syam Shaiju khalid Midhun manual,0
4,J A K E S. B EJ O Y !!!,0
5,Pwoli item padam kananda ennu karuthiyatha pak...,0
6,Oru ratchasan feel kittitu ullalo. Bgm athra a...,0
7,Super casting,0
8,ഷൈലോക്ക് ന്റെ നല്ല ടീസർ ആയിട്ട് പോലും ട്രോളി ന...,0
9,ന്റമ്മോ ഇജ്ജാതി trailor.. Uffftrailor ethegilu...,0


In [6]:
import sys
sys.path.append("/content/drive/MyDrive/Capstone/Translator/")
from google_trans_new import google_translator

In [7]:
translator = google_translator()
for index, row in df_train.iterrows(): # Check if emoji is contained in text
    text = str(row['text'])
    found = demoji.findall(text)
    if found:
        # There is an emoji now detect language
        if detect(text) == 'ml': # Text is Malayalam
            for key, value in found.items():
                emojiEng = value
                # Translate to Malayalam
                transText = translator.translate(emojiEng, lang_tgt='ml', lang_src='en')
                text = text.replace(key, transText + ' ')
        else: # Text is english
            for key, value in found.items():
                text = text.replace(key, value + ' ')

        df_train.at[index, 'text'] = text

In [8]:
df_train.head(30)

,text,label
0,പലദേശം. പല ഭാഷ ഒരേ ഒരു രാജാവ് അല്ലാതെ സ്വന്ത...,0
1,ഈ ഓണം ഏട്ടനും പിള്ളേർക്ക് ഉള്ളതാണ് എന്ന് ഉള്ളവ...,0
2,ആരണ്ട ആരണ്ട തലുണ്ടാകാണാ ആരണ്ട ഞാൻ ആണ്ട ഞാൻ ആണ്...,0
3,Sushin syam Shaiju khalid Midhun manual,0
4,J A K E S. B EJ O Y !!!,0
5,Pwoli item padam kananda ennu karuthiyatha pak...,0
6,Oru ratchasan feel kittitu ullalo. Bgm athra a...,0
7,Super casting,0
8,ഷൈലോക്ക് ന്റെ നല്ല ടീസർ ആയിട്ട് പോലും ട്രോളി ന...,0
9,ന്റമ്മോ ഇജ്ജാതി trailor.. Uffftrailor ethegilu...,0


In [ ]:
for index, row in df_train.iterrows():
    found = demoji.findall(row['text'])
    if found:
        print('Found exception')

In [9]:
import contractions
# contracted text
for index, row in df_train.iterrows():# creating an empty list
    expanded_words = []
    for word in str(row['text']).split():# using contractions.fix to expand the shotened words
        expanded_words.append(contractions.fix(word))

    expanded_text = ' '.join(expanded_words)
    df_train.at[index, 'text'] = expanded_text


for index, row in df_train.iterrows():
    Sentence = str(row['text'])
    for i in Sentence:
        # checking whether the char is punctuation.
        if i in string.punctuation:
            Sentence = Sentence.replace(i, '')
    df_train.at[index, 'text'] = Sentence.lower()

In [10]:
df_train.head(30)

,text,label
0,പലദേശം പല ഭാഷ ഒരേ ഒരു രാജാവ് അല്ലാതെ സ്വന്തം ര...,0
1,ഈ ഓണം ഏട്ടനും പിള്ളേർക്ക് ഉള്ളതാണ് എന്ന് ഉള്ളവ...,0
2,ആരണ്ട ആരണ്ട തലുണ്ടാകാണാ ആരണ്ട ഞാൻ ആണ്ട ഞാൻ ആണ്...,0
3,sushin syam shaiju khalid midhun manual,0
4,j a k e s b ej o y,0
5,pwoli item padam kananda ennu karuthiyatha pak...,0
6,oru ratchasan feel kittitu ullalo bgm athra ad...,0
7,super casting,0
8,ഷൈലോക്ക് ന്റെ നല്ല ടീസർ ആയിട്ട് പോലും ട്രോളി ന...,0
9,ന്റമ്മോ ഇജ്ജാതി trailor uffftrailor ethegilum ...,0


In [11]:
malayalam_stopwords = [
    "അത്", "ഇത്", "അവന്", "അവള്", "എന്റെ",
    "എന്നേക്കും", "എന്നേക്കും", "എന്നു", "എന്നും", "എന്ന്",
    "എന്നുമുണ്ട്", "എന്നുമ്", "എന്നുമ്", "എന്നും", "എന്നൈക്കൊണ്ട്",
    "എന്നും", "എന്നുമ്", "എന്നേക്കൊണ്ട്", "ഏത്", "ഒരു",
    "ഒരുവർ", "ഒരുവളുകൾ", "എന്ന", "എന്നത്",
    "എന്നതായി", "എന്നാണ്", "എന്നാൽ", "എന്നോ", "എന്നോട്",
    "എന്ന്", "ഇനി", "ഇപ്പോൾ", "ഇതിൽ", "അല്ല",
    "ഇതുമുതൽ", "ഇതിൽ", "എന്നുകൊണ്ട്", "ഇതുപോലെ", "അതു",
    "ഇതിനെ", "ഇതാ", "അതാ", "അതായത്",
    "ഇതായത്", "ഇതിന്റെ", "അതിന്റെ", "ഇതിൽ", "ഇന്ന്",
    "ഇന്നത്തെ", "ഇന്നു", "ഇന്നുകഴിഞ്ഞ്", "ഇന്നുകൊണ്ട്", "ഇന്നുവരെ",
    "ഇന്നുവർ"
]


In [12]:
data = []

for index, row in df_train.iterrows():
    sent = ''
    line = str(row['text'])
    for i in line.split():
        if i not in malayalam_stopwords:
            sent += i + " "
    data.append({'text': sent, 'label': str(row['label'])})

new_df_train = pd.DataFrame(data)
new_df_train.head(30)


,text,label
0,പലദേശം പല ഭാഷ ഒരേ രാജാവ് അല്ലാതെ സ്വന്തം രാജവയത്,0
1,ഈ ഓണം ഏട്ടനും പിള്ളേർക്ക് ഉള്ളതാണ് ഉള്ളവർ ലൈക്...,0
2,ആരണ്ട ആരണ്ട തലുണ്ടാകാണാ ആരണ്ട ഞാൻ ആണ്ട ഞാൻ ആണ്...,0
3,sushin syam shaiju khalid midhun manual,0
4,j a k e s b ej o y,0
5,pwoli item padam kananda ennu karuthiyatha pak...,0
6,oru ratchasan feel kittitu ullalo bgm athra ad...,0
7,super casting,0
8,ഷൈലോക്ക് ന്റെ നല്ല ടീസർ ആയിട്ട് പോലും ട്രോളി ന...,0
9,ന്റമ്മോ ഇജ്ജാതി trailor uffftrailor ethegilum ...,0


In [13]:
!pip install -qq indic-nlp-library
!pip install -qq tqdm
from indicnlp import common
from indicnlp import loader
from indicnlp.script import indic_scripts
from indicnlp.tokenize import indic_tokenize
from indicnlp.morph import unsupervised_morph
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
INDIC_NLP_RESOURCES = r"/content/indic_nlp_resources"
common.set_resources_path(INDIC_NLP_RESOURCES)
loader.load()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 16.5 MB/s eta 0:00:00
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 23.91 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (28/28), done.


In [14]:
analyzer = unsupervised_morph.UnsupervisedMorphAnalyzer('ml')

In [15]:
from langdetect import detect, LangDetectException
from tqdm import tqdm

def stem_malayalam_text(text):
    tokens = text.split()
    stemmed_tokens = []
    for token in tokens:
        analyzed_tokens = analyzer.morph_analyze_document([token])
        if analyzed_tokens:
            stemmed_tokens.append(analyzed_tokens[0])
        else:
            stemmed_tokens.append(token)  # If stemming fails, keep the original token
    return ' '.join(stemmed_tokens)


for index, row in tqdm(new_df_train.iterrows(), total=len(new_df_train)):
    text = str(row['text'])

    try:
        detected_lang = detect(text)
    except LangDetectException as e:
        # Handle the exception, for example by setting the detected_lang to None
        detected_lang = None

    if detected_lang == 'ml':
        stemmed_text = stem_malayalam_text(text)
        new_df_train.at[index, 'text'] = stemmed_text


print(new_df_train.head(20))


100%|██████████| 16010/16010 [03:22<00:00, 78.93it/s]

                                                 text label
0            പല പല ഭാഷ ഒരേ രാജാവ് അല്ലാതെ സ്വന്തം രാജ     0
1                    ഈ ഓണ ഏ പിള്ള ഉള്ള ഉള്ള ലൈക്‌ അടി     0
2   ആ ആ തല ആ ഞാ ആ ഞാ ആ ഞാ royal mech ആ ആ ആ മീശ പ ആ...     0
3            sushin syam shaiju khalid midhun manual      0
4                                 j a k e s b ej o y      0
5   pwoli item padam kananda ennu karuthiyatha pak...     0
6   oru ratchasan feel kittitu ullalo bgm athra ad...     0
7                                      super casting      0
8   ഷൈല ന്റെ നല്ല ടീ ആയിട്ട പോലും ട്രോളി നടന്ന ല ഫ...     0
9   ന്റമ്മോ ഇജ്ജാതി trailor uffftrailor ethegilum ...     0
10                        ബോംബ ആകു തോന്നുന്ന like അടി     0
11   ee asurante varavin katta waitingmammookka uyir      0
12  rajuvettan chunkalla chunkidipaan adi makalle ...     0
13  ettan kurch koode perfect akum enn vicharikunu...     0
14                     മാമാങ്ക മമ്മൂക്ക കണ്ടു ഇതി ആരാ     0
15  vada ithu kettu romancham vannavarku

In [16]:
new_df_train['text'].isnull().sum()

0

In [17]:
new_df_train.to_csv('malayalam_dataset_1_train.csv', index=False)

In [18]:
df_test = pd.DataFrame(dataset["validation"])
label_mapping = {
    0: 'Not_offensive',
    1: 'Offensive',
    2: 'Offensive',
    3: 'Offensive',
    4: 'Offensive',
    5: 'not-Malayalam',
}
df_test['label'].replace(label_mapping, inplace=True)

label_mapping1 = {
    'Not_offensive':0,
    'Offensive':1,
    'not-Malayalam':2,
}
df_test['label'].replace(label_mapping1, inplace=True)
print(df_test.info())
label_counts = df_test['label'].value_counts()
print(label_counts)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    1999 non-null   object
 1   label   1999 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 31.4+ KB
None
0    1779
2     163
1      57
Name: label, dtype: int64


In [19]:
df_test.head(20)

,text,label
0,Gopichettante BGM um mammookayum ishtapedunnav...,0
1,ഇത് ഒരു പെണ്ണ് തന്നെ ആണോ direct ചെയ്യുന്നത് p...,0
2,പൃഥ്വിരാജ് സുരാജേട്ടൻ ലാലും അലക്സ്.. Lal jr. ന...,0
3,പോകരുത് മക്കളെ പോക്ക .......... നൻ കണ്ട് എന്റ്...,0
4,avan Varum ente Makan Madhura Raja....,0
5,you guys dub in kannada we are kannadigas alwa...,2
6,Oru nalla cinema kkaayi kathirikkkunnu. Dha...,0
7,Nine film arkum venda,0
8,ഭവനേ. കൊലേ കാട്ടിലേ ഇരിക്ക് കൊല്ലാമ വിടമാട്ടെ ...,0
9,Mathew Thiruvankaulam...chekkan pwoli.anu....n...,0


In [20]:
translator = google_translator()
for index, row in df_test.iterrows(): # Check if emoji is contained in text
    text = str(row['text'])
    found = demoji.findall(text)
    if found:
        # There is an emoji now detect language
        if detect(text) == 'ml': # Text is Malayalam
            for key, value in found.items():
                emojiEng = value
                # Translate to Malayalam
                transText = translator.translate(emojiEng, lang_tgt='ml', lang_src='en')
                text = text.replace(key, transText + ' ')
        else: # Text is english
            for key, value in found.items():
                text = text.replace(key, value + ' ')

        df_test.at[index, 'text'] = text

In [21]:
df_test

,text,label
0,Gopichettante BGM um mammookayum ishtapedunnav...,0
1,ഇത് ഒരു പെണ്ണ് തന്നെ ആണോ direct ചെയ്യുന്നത് p...,0
2,പൃഥ്വിരാജ് സുരാജേട്ടൻ ലാലും അലക്സ്.. Lal jr. ന...,0
3,പോകരുത് മക്കളെ പോക്ക .......... നൻ കണ്ട് എന്റ്...,0
4,avan Varum ente Makan Madhura Raja....,0
...,...,...
1994,Onnam ഇട്ടി മണി & brathors ഡേ വിന്നർ ലാലേട്ടൻ ...,0
1995,Manju chechi undel hero vendaa. Chechi vere le...,0
1996,Ith kollaaaam....kanda ooodayp trailer ne kalu...,0
1997,Ufffff. Pwoli lalattaa... Katta waiting,0


In [22]:
for index, row in df_test.iterrows():
    found = demoji.findall(row['text'])
    if found:
        print('Found exception')

In [23]:
import contractions
# contracted text
for index, row in df_test.iterrows():# creating an empty list
    expanded_words = []
    for word in str(row['text']).split():# using contractions.fix to expand the shotened words
        expanded_words.append(contractions.fix(word))

    expanded_text = ' '.join(expanded_words)
    df_test.at[index, 'text'] = expanded_text


for index, row in df_test.iterrows():
    Sentence = str(row['text'])
    for i in Sentence:
        # checking whether the char is punctuation.
        if i in string.punctuation:
            Sentence = Sentence.replace(i, '')
    df_test.at[index, 'text'] = Sentence.lower()

In [24]:
df_test

,text,label
0,gopichettante bgm um mammookayum ishtapedunnav...,0
1,ഇത് ഒരു പെണ്ണ് തന്നെ ആണോ direct ചെയ്യുന്നത് po...,0
2,പൃഥ്വിരാജ് സുരാജേട്ടൻ ലാലും അലക്സ് lal jr ന്റെ...,0
3,പോകരുത് മക്കളെ പോക്ക നൻ കണ്ട് എന്റ്യെ അമ്മോ പ...,0
4,avan varum ente makan madhura raja,0
...,...,...
1994,onnam ഇട്ടി മണി brathors ഡേ വിന്നർ ലാലേട്ടൻ ര...,0
1995,manju chechi undel hero vendaa chechi vere lev...,0
1996,ith kollaaaamkanda ooodayp trailer ne kalum ok...,0
1997,ufffff pwoli lalattaa katta waiting,0


In [25]:
data = []

for index, row in df_test.iterrows():
    sent = ''
    line = str(row['text'])
    for i in line.split():
        if i not in malayalam_stopwords:
            sent += i + " "
    data.append({'text': sent, 'label': str(row['label'])})

new_df_test = pd.DataFrame(data)
new_df_test.head(30)


,text,label
0,gopichettante bgm um mammookayum ishtapedunnav...,0
1,പെണ്ണ് തന്നെ ആണോ direct ചെയ്യുന്നത് polichu ni...,0
2,പൃഥ്വിരാജ് സുരാജേട്ടൻ ലാലും അലക്സ് lal jr ന്റെ...,0
3,പോകരുത് മക്കളെ പോക്ക നൻ കണ്ട് എന്റ്യെ അമ്മോ പോ...,0
4,avan varum ente makan madhura raja,0
5,you guys dub in kannada we are kannadigas alwa...,2
6,oru nalla cinema kkaayi kathirikkkunnu dhayava...,0
7,nine film arkum venda,0
8,ഭവനേ കൊലേ കാട്ടിലേ ഇരിക്ക് കൊല്ലാമ വിടമാട്ടെ t...,0
9,mathew thiruvankaulamchekkan pwolianunlnammade...,0


In [26]:
from langdetect import detect, LangDetectException
from tqdm import tqdm

def stem_malayalam_text(text):
    tokens = text.split()
    stemmed_tokens = []
    for token in tokens:
        analyzed_tokens = analyzer.morph_analyze_document([token])
        if analyzed_tokens:
            stemmed_tokens.append(analyzed_tokens[0])
        else:
            stemmed_tokens.append(token)  # If stemming fails, keep the original token
    return ' '.join(stemmed_tokens)


for index, row in tqdm(new_df_test.iterrows(), total=len(new_df_test)):
    text = str(row['text'])

    try:
        detected_lang = detect(text)
    except LangDetectException as e:
        # Handle the exception, for example by setting the detected_lang to None
        detected_lang = None

    if detected_lang == 'ml':
        stemmed_text = stem_malayalam_text(text)
        new_df_test.at[index, 'text'] = stemmed_text


print(new_df_test.head(20))


100%|██████████| 1999/1999 [00:24<00:00, 80.61it/s] 

                                                 text label
0   gopichettante bgm um mammookayum ishtapedunnav...     0
1       പെണ്ണ തന്നെ ആണ direct ചെയ്യുന്ന polichu nivin     0
2   പൃഥ്വി സു ല അ lal jr ന്റെ വെറൈറ്റി ചിത്രം പ്രത...     0
3    പോകരുത് മ പോക്ക ന കണ്ട് എ അമ്മ പോളി മറക്ക ഒരിക്ക     0
4                 avan varum ente makan madhura raja      0
5   you guys dub in kannada we are kannadigas alwa...     2
6   oru nalla cinema kkaayi kathirikkkunnu dhayava...     0
7                              nine film arkum venda      0
8   ഭവന കൊല കാട്ടി ഇരിക്ക കൊല്ല വിട the boss is ba...     0
9   mathew thiruvankaulamchekkan pwolianunlnammade...     0
10                     neega vaa thalaivare megastar      0
11     cinema iragiyittum view cheyyunnavar like adi      0
12  sunny akante song ennu youtubeil varunno annu ...     0
13  ettimani thakarkkum lalettan polichadukkum ona...     0
14  sharukhante fan movie pole ulla script aanenn ...     0
15  film kande kazhinnitte trailer onnoo

In [27]:
new_df_test['text'].isnull().sum()

0

In [28]:
new_df_test.to_csv('malayalam_dataset_1_test.csv', index=False)